A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Load and Analyse dataset

Read in the csv file rental_info.csv using pandas.

In [2]:
df = pd.read_csv('rental_info.csv')

print(df.head)
print(df.info())

<bound method NDFrame.head of                      rental_date                return_date  amount  \
0      2005-05-25 02:54:33+00:00  2005-05-28 23:40:33+00:00    2.99   
1      2005-06-15 23:19:16+00:00  2005-06-18 19:24:16+00:00    2.99   
2      2005-07-10 04:27:45+00:00  2005-07-17 10:11:45+00:00    2.99   
3      2005-07-31 12:06:41+00:00  2005-08-02 14:30:41+00:00    2.99   
4      2005-08-19 12:30:04+00:00  2005-08-23 13:35:04+00:00    2.99   
...                          ...                        ...     ...   
15856  2005-08-22 10:49:15+00:00  2005-08-29 09:52:15+00:00    6.99   
15857  2005-07-31 09:48:49+00:00  2005-08-04 10:53:49+00:00    4.99   
15858  2005-08-20 10:35:30+00:00  2005-08-29 13:03:30+00:00    8.99   
15859  2005-07-31 13:10:20+00:00  2005-08-08 14:07:20+00:00    7.99   
15860  2005-08-18 06:33:55+00:00  2005-08-24 07:14:55+00:00    5.99   

       release_year  rental_rate  length  replacement_cost  \
0            2005.0         2.99   126.0             16

In [3]:
print(df.isna().sum())

rental_date         0
return_date         0
amount              0
release_year        0
rental_rate         0
length              0
replacement_cost    0
special_features    0
NC-17               0
PG                  0
PG-13               0
R                   0
amount_2            0
length_2            0
rental_rate_2       0
dtype: int64


Create a column named "rental_length_days" using the columns "return_date" and "rental_date", and add it to the pandas DataFrame. This column should contain information on how many days a DVD has been rented by a customer.

In [4]:
print(df[['return_date','rental_date']].head())

                 return_date                rental_date
0  2005-05-28 23:40:33+00:00  2005-05-25 02:54:33+00:00
1  2005-06-18 19:24:16+00:00  2005-06-15 23:19:16+00:00
2  2005-07-17 10:11:45+00:00  2005-07-10 04:27:45+00:00
3  2005-08-02 14:30:41+00:00  2005-07-31 12:06:41+00:00
4  2005-08-23 13:35:04+00:00  2005-08-19 12:30:04+00:00


In [5]:
# Convert dates to datetime and extract the date
df['return_date'] = pd.to_datetime(df['return_date'])
df['rental_date'] = pd.to_datetime(df['rental_date'])

In [6]:
# Add new column to extract total days a DVD is rented
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

In [7]:
# Verify the column
print(df['rental_length_days'].head())

0    3
1    2
2    7
3    2
4    4
Name: rental_length_days, dtype: int64


### Extract new features

Create two columns of dummy variables from "special_features", which takes the value of 1 when:
- The value is "Deleted Scenes", storing as a column called "deleted_scenes".
- The value is "Behind the Scenes", storing as a column called "behind_the_scenes".


In [8]:
print(df['special_features'].unique())

['{Trailers,"Behind the Scenes"}' '{Trailers}'
 '{Commentaries,"Behind the Scenes"}' '{Trailers,Commentaries}'
 '{"Deleted Scenes","Behind the Scenes"}'
 '{Commentaries,"Deleted Scenes","Behind the Scenes"}'
 '{Trailers,Commentaries,"Deleted Scenes"}' '{"Behind the Scenes"}'
 '{Trailers,"Deleted Scenes","Behind the Scenes"}'
 '{Commentaries,"Deleted Scenes"}' '{Commentaries}'
 '{Trailers,Commentaries,"Behind the Scenes"}'
 '{Trailers,"Deleted Scenes"}' '{"Deleted Scenes"}'
 '{Trailers,Commentaries,"Deleted Scenes","Behind the Scenes"}']


In [9]:
import numpy as np

# Create columns deleted_scenes and behind_the_scenes
df['deleted_scenes'] = np.where(df['special_features'].str.contains('Deleted Scenes'), 1, 0)
df['behind_the_scenes'] = np.where(df['special_features'].str.contains('Behind the Scenes'), 1, 0)

In [10]:
# Verify the columns
print(df[['deleted_scenes','behind_the_scenes']])

       deleted_scenes  behind_the_scenes
0                   0                  1
1                   0                  1
2                   0                  1
3                   0                  1
4                   0                  1
...               ...                ...
15856               1                  1
15857               1                  1
15858               1                  1
15859               1                  1
15860               1                  1

[15861 rows x 2 columns]


In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   rental_date         15861 non-null  datetime64[ns, UTC]
 1   return_date         15861 non-null  datetime64[ns, UTC]
 2   amount              15861 non-null  float64            
 3   release_year        15861 non-null  float64            
 4   rental_rate         15861 non-null  float64            
 5   length              15861 non-null  float64            
 6   replacement_cost    15861 non-null  float64            
 7   special_features    15861 non-null  object             
 8   NC-17               15861 non-null  int64              
 9   PG                  15861 non-null  int64              
 10  PG-13               15861 non-null  int64              
 11  R                   15861 non-null  int64              
 12  amount_2            15861 non-nu

### Split data

Make a pandas DataFrame called X containing all the appropriate features you can use to run the regression models, avoiding columns that leak data about the target.

Choose the "rental_length_days" as the target column and save it as a pandas Series called y.


In [12]:
from sklearn.model_selection import train_test_split

# Identify the columns to drop
cols_to_drop = ['rental_length_days', 'rental_date', 'return_date', 'special_features']

X = df.drop(columns=cols_to_drop) # features
y = df['rental_length_days'] # target

In [13]:
# Verify the shape
print(X.shape)
print(y.shape)

(15861, 14)
(15861,)


Split the data into X_train, y_train, X_test, and y_test train and test sets, avoiding any features that leak data about the target variable, and include 20% of the total data in the test set.

Set random_state to 9 whenever you use a function/method involving randomness, for example, when doing a test-train split.

In [14]:
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=9)

In [15]:
from sklearn.preprocessing import StandardScaler

# Scale features after splitting
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [16]:
from sklearn.linear_model import Lasso

# Initialize Lasso
lasso = Lasso(alpha=0.01, random_state=9)

# Fit Lasso regression
lasso.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=9)

### Identify important features

In [17]:
# Get the coefficients
lasso_coef = lasso.coef_

# Display coefficients with corresponding features
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lasso_coef
}).sort_values(by='Coefficient', ascending=False)

print(feature_importance)

              Feature  Coefficient
0              amount     3.342176
5               NC-17     0.043781
3              length     0.038711
7               PG-13     0.034685
13  behind_the_scenes     0.030842
6                  PG     0.029347
1        release_year     0.021844
10           length_2     0.000000
4    replacement_cost    -0.012465
8                   R    -0.025077
12     deleted_scenes    -0.055960
11      rental_rate_2    -0.163977
9            amount_2    -0.592833
2         rental_rate    -1.797527


In [18]:
# Identify features with non-zero coefficients
important_features = X.columns[lasso_coef > 0].tolist()
print(important_features)

['amount', 'release_year', 'length', 'NC-17', 'PG', 'PG-13', 'behind_the_scenes']


In [19]:
# Subset training and test data to include only these features
X_train_important = X_train[important_features]
X_test_important = X_test[important_features]

print(X_train_important.shape)
print(X_test_important.shape)

(12688, 7)
(3173, 7)


### Modeling

Recommend a model yielding a mean squared error (MSE) less than 3 on the test set

Save the model you would recommend as a variable named best_model, and save its MSE on the test set as best_mse.


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Define models 
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=9, n_estimators=100),
    'Decision Tree': DecisionTreeRegressor(random_state=9, max_depth=10)
}

In [21]:
for name, model in models.items():

    # Train and predict
    model.fit(X_train_important, y_train)
    y_pred = model.predict(X_test_important)
    
    # Evaluate model
    mse = round(mean_squared_error(y_test, y_pred),2)

    # Append into results
    print(f'Model: {name}')
    print(f'Test MSE score: {mse}')

Model: Linear Regression
Test MSE score: 4.85
Model: Random Forest
Test MSE score: 2.34
Model: Decision Tree
Test MSE score: 3.74


Since Random Forest have test MSE score less than 3, we are going to pick this model to do further tuning.

### Hyperparameter Tuning

In [24]:
from sklearn.model_selection import RandomizedSearchCV

# Define parameter grid
param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize Random Forest
model = RandomForestRegressor(random_state=9)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5,
    random_state=9,
    n_jobs=-1,
    verbose=2
)

# Fit the random search
random_search.fit(X_train_important, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Predict
y_pred = best_model.predict(X_test_important)

# Evaluate the best model
best_mse = round(mean_squared_error(y_test, y_pred),2)

print(f'Best parameters: {random_search.best_params_}')
print(f'Test MSE score: {mse}')


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': True}
Test MSE score: 2.33
